In [32]:
import datetime, time, math

In [5]:
eth_wallet = '0x154a22e768fcb0098162D5f9B5A4DB3ce628b03E'

In [33]:
Wallet_Address = '0x488011d9f80e6ea47ab79a531464c6118e37bf6e'
Account_ = '0x94db19933b524c2624c9216ed3e0fef400876b3bd1756210822a2254fe7f1414'
Orderly_API_Key = 'ed25519:ABcMjrBAi8guUxE6cjpGadnHdhUXd4bwwM8SJMx2zsnz'
Orderly_API_Secret = 'ed25519:8M1oRWsgwyUVhC881pPSLg6KvtBZZs1qZr4CYMNvutnM'

## check if account exists or not

In [10]:
import requests

url = "https://testnet-api-evm.orderly.network/v1/get_account"

querystring = {"address": Wallet_Address, 'broker_id':'woofi_dex' }

response = requests.request("GET", url, params=querystring)

print(response.text)

{"success":false,"code":-1607,"message":"account does not exist"}


## generate keccak hash keys

In [12]:
import sha3
k = sha3.keccak_256()
k.update(b'age')
print(k.hexdigest())

36383cc9cfbf1dc87c78c2529ae2fcd4e3fc4e575e154b357ae3a8b2739113cf


## get list of brokers

In [28]:
import requests
import pandas as pd

url = "https://api-evm.orderly.org/v1/public/broker/name"

response = requests.request("GET", url)

print(response.text)

data = response.text
pd.DataFrame(response.json()['data']['rows'])

{"success":true,"data":{"rows":[{"broker_id":"woofi_pro","broker_name":"WOOFi Pro"},{"broker_id":"orderly","broker_name":"Orderly"},{"broker_id":"0xfin","broker_name":"0xfin"},{"broker_id":"busywhale","broker_name":"BusyWhale"},{"broker_id":"logx","broker_name":"LogX Aggregator "},{"broker_id":"emdx_dex","broker_name":"EMDX"},{"broker_id":"bitoro_network","broker_name":"Bitoro Network"},{"broker_id":"quick_perps","broker_name":"QuickSwap"},{"broker_id":"empyreal","broker_name":"Empyreal"}]},"timestamp":1710769072151}


,broker_id,broker_name
0,woofi_pro,WOOFi Pro
1,orderly,Orderly
2,0xfin,0xfin
3,busywhale,BusyWhale
4,logx,LogX Aggregator
5,emdx_dex,EMDX
6,bitoro_network,Bitoro Network
7,quick_perps,QuickSwap
8,empyreal,Empyreal


## authenticate account

In [49]:
import json
import os
from base58 import b58decode
from requests import Request, Session

from cryptography.hazmat.primitives.asymmetric.ed25519 import Ed25519PrivateKey

from signer import Signer

live_url = 'https://api-evm.orderly.org'



base_url = "https://testnet-api-evm.orderly.org"

orderly_account_id = Account_

key = b58decode(Orderly_API_Secret.split(':')[1])
orderly_key = Ed25519PrivateKey.from_private_bytes(key)

session = Session()
signer = Signer(orderly_account_id, orderly_key)

req = signer.sign_request(
    Request(
        "POST",
        "%s/v1/order" % live_url,
        json={
            "symbol": "PERP_ETH_USDC",
            "order_type": "MARKET",
            "order_quantity": 0.01,
            "side": "BUY",
        },
    )
)
res = session.send(req)
response = json.loads(res.text)

auth_data = response

{'orderly-timestamp': '1710770687548', 'orderly-account-id': '0x94db19933b524c2624c9216ed3e0fef400876b3bd1756210822a2254fe7f1414', 'orderly-key': 'ed25519:ABcMjrBAi8guUxE6cjpGadnHdhUXd4bwwM8SJMx2zsnz', 'orderly-signature': 'RiY9BHcyqGOtNSrflj5pwwVntzWbZHxR1Rj8VfgZ8KSvmjOqmWwBHKW7bmzKvgibQ2zzbYp7TVAlE_GPsTGvCg=='}


In [50]:
auth_data

{'success': True,
 'data': {'order_id': 404967936,
  'client_order_id': None,
  'order_type': 'MARKET',
  'order_price': None,
  'order_quantity': 0.01,
  'order_amount': None},
 'timestamp': 1710770687728}

## generate nounce value - valid for 2 mins only

In [30]:
import requests

url = "https://api-evm.orderly.org/v1/registration_nonce"

response = requests.request("GET", url)

print(response.text)

{"success":true,"data":{"registration_nonce":"105236611141"},"timestamp":1710769213367}


In [45]:
from datetime import datetime
import json
import math
import os
import requests

from eth_account import Account, messages


MESSAGE_TYPES = {
    "EIP712Domain": [
        {"name": "name", "type": "string"},
        {"name": "version", "type": "string"},
        {"name": "chainId", "type": "uint256"},
        {"name": "verifyingContract", "type": "address"},
    ],
    "Registration": [
        {"name": "brokerId", "type": "string"},
        {"name": "chainId", "type": "uint256"},
        {"name": "timestamp", "type": "uint64"},
        {"name": "registrationNonce", "type": "uint256"},
    ],
}

OFF_CHAIN_DOMAIN = {
    "name": "Orderly",
    "version": "1",
    "chainId": 421614,
    "verifyingContract": "0xCcCCccccCCCCcCCCCCCcCcCccCcCCCcCcccccccC",
}

base_url = live_url # "https://testnet-api-evm.orderly.org"
broker_id = "woofi_pro"
chain_id = 421614

account: Account = Account.from_key(auth_data['orderly-signature'])

res = requests.get("%s/v1/registration_nonce" % base_url)
response = json.loads(res.text)
registration_nonce = response["data"]["registration_nonce"]

d = datetime.utcnow()
epoch = datetime(1970, 1, 1)
timestamp = math.trunc((d - epoch).total_seconds() * 1_000)

register_message = {
    "brokerId": broker_id,
    "chainId": chain_id,
    "timestamp": timestamp,
    "registrationNonce": registration_nonce,
}

encoded_data = messages.encode_typed_data(
    domain_data=OFF_CHAIN_DOMAIN,
    message_types={"Registration": MESSAGE_TYPES["Registration"]},
    message_data=register_message,
)
signed_message = account.sign_message(encoded_data)

res = requests.post(
    "%s/v1/register_account" % base_url,
    headers={"Content-Type": "application/json"},
    json={
        "message": register_message,
        "signature": signed_message.signature.hex(),
        "userAddress": account.address,
    },
)
response = json.loads(res.text)

orderly_account_id = response["data"]["account_id"]
print("orderly_account_id:", orderly_account_id)


KeyError: 'orderly-signature'

In [41]:
val = 'ant'.encode('utf-8').hex()
# hex(int(val))
val

'616e74'